In [ ]:
!pip install -q langchain==0.2.16
!pip install -q ragas==0.1.19

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
# logging.langsmith("rag_evaluation")

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import csv

file_path = "../data/csv_data/rental_data_with_null.csv"


def get_csv_headers(file_path):
    with open(file_path, mode="r", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # 첫 번째 줄(헤더) 가져오기
    return headers


headers = get_csv_headers(file_path)

# CSV 로더 생성
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": headers,
    },
    # source_column="place",
    content_columns=headers,
    metadata_columns=[
        "place",
        "oneroom_half_year",
        "oneroom_year",
        "tworoom_half_year",
        "tworoom_year",
    ],
)
docs = loader.load()
print(docs[1].metadata)

In [ ]:
# metadata 값을 integer로 변환하는 함수
def convert_metadata_to_int(docs, columns_to_convert):
    """
    문서 리스트의 metadata 값을 integer로 변환하는 함수.
    
    Args:
        docs: CSVLoader에서 로드된 문서 리스트.
        columns_to_convert: integer로 변환할 열 이름 리스트.

    Returns:
        변환된 문서 리스트.
    """
    for doc in docs:
        for column in columns_to_convert:
            if column in doc.metadata:
                try:
                    # metadata 값을 integer로 변환
                    doc.metadata[column] = float(doc.metadata[column])
                except ValueError:
                    # 변환 실패 시 NaN 또는 다른 처리 (여기서는 0으로 설정)
                    doc.metadata[column] = float('nan')
    return docs

# 변환할 열 이름 리스트
columns_to_convert = [
    "oneroom_half_year",
    "oneroom_year",
    "tworoom_half_year",
    "tworoom_year",
]

# metadata 값을 integer로 변환
docs = convert_metadata_to_int(docs, columns_to_convert)

In [ ]:
i = 1

for doc in docs[i:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>\n\n"

    docs[i].page_content = row_str
    i += 1
    # print(ret[i].page_content)
    # ret += row_str

ret = docs

In [ ]:
# 각 문서에 파일 이름을 추가합니다.
for i in ret[1:]:
    i.metadata["filename"] = i.metadata["source"]

In [ ]:
ret[1].metadata

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [ ]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

In [ ]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

test_df=testset.to_pandas()

# DataFrame을 CSV 파일로 저장
test_df.to_csv("../data/csv_data/ragas_synthetic_dataset.csv", index=False)

In [14]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the contact details and price range f...,[<row><name>애플하우스</name><address>호서로125번길 62-3...,The contact details for the accommodation at '...,simple,[{'source': '../data/csv_data/rental_data_with...,True
1,What is the phone number for the property loca...,[<row><name>에코빌</name><address>호서로125번길 62-7</...,The phone number for the property located at 1...,simple,[{'source': '../data/csv_data/rental_data_with...,True
2,What is the price for renting the Ritz House l...,[<row><name>리치하우스</name><address>호서로 125번길 22-...,The price for renting the Ritz House located a...,simple,[{'source': '../data/csv_data/rental_data_with...,True
3,What is the price for renting the 리치하우스 for on...,[<row><name>리치하우스</name><address>호서로 125번길 22-...,The price for renting the 리치하우스 for one year i...,simple,[{'source': '../data/csv_data/rental_data_with...,True
4,What's the price range for the property at 125...,[<row><name>벧엘원룸</name><address>호서로125번길 40-4 ...,The price range for the property at 125 Haeore...,reasoning,[{'source': '../data/csv_data/rental_data_with...,True


In [15]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 5
})